In [24]:
import sys
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn import set_config; set_config(display='diagram')

sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")
from preprocessing.preprocess_1_cleaning import *
from preprocessing.preprocess_2_features import *
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer
import datetime
from sklearn.preprocessing import MultiLabelBinarizer



In [17]:
def only_main_games(df):
    main_game_mask =df['category'] == 'main'
    only_main_games = df[main_game_mask]
    
    return only_main_games.drop('category', axis=1)


def create_gen(seri):
    gen_1_end = datetime.datetime.strptime('1983-01-01', '%Y-%m-%d')
    gen_2_end = datetime.datetime.strptime('1995-01-01', '%Y-%m-%d')
    gen_3_end = datetime.datetime.strptime('2006-01-01', '%Y-%m-%d')
    gen_4_end = datetime.datetime.strptime('2014-01-01', '%Y-%m-%d')
    gen_5_end = datetime.datetime.strptime('2023-10-01', '%Y-%m-%d')
    
    dupe = seri
    dupe['gen'] = None

    for i, date in enumerate(dupe['release_date']):
        if date < gen_1_end:
            dupe.iloc[i, 27] = 1
        elif date < gen_2_end:
            dupe.iloc[i, 27] = 2
        elif date < gen_3_end:
            dupe.iloc[i, 27] = 3
        elif date < gen_4_end:
            dupe.iloc[i, 27] = 4
        elif date < gen_5_end:
            dupe.iloc[i, 27] = 5
        else:
            dupe.iloc[i, 27] = 6
    return dupe


In [2]:
data = pd.read_csv('../raw_data/all_game_data_v1_corrected2', low_memory=False)

In [3]:
testing_df = cleaning_in_notebook(data)

/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column]=pd.to_datetime(df[column])
/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_2_features.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupe['gen'] = None
/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [4]:
testing_df

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,image,url,gen
0,PilotXross,2030-12-20,1,0,2,2,[PilotXross],0.0,"[Indie, Racing, Sport]",[SteamVR],...,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/pilotxross/,6
1,Plague Hunter,2029-02-07,0,0,2,7,[],0.0,"[Adventure, Indie]",[Windows PC],...,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/plague-hunter/,6
2,The God Slayer,2027-12-31,0,0,1,8,[Pathea Games],0.0,[RPG],[PlayStation 5],...,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/the-god-slayer/,6
3,Tales of the Death,2025-12-31,0,0,0,4,"[8 Million Bit, Hanoi Studios]",0.0,[],[Windows PC],...,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/tales-of-the-d...,6
4,Riversiders,2025-12-31,1,0,0,0,[],0.0,"[Adventure, Racing]",[Windows PC],...,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/riversiders/,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119538,All For Nought: Tic Tac Toe,2020-09-03,0,0,2,0,[],0.0,[Strategy],[Windows PC],...,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/all-for-nought...,5
119548,Siren Head: The Siren's Forest,2020-10-15,4,0,1,0,[],0.0,[Indie],[Windows PC],...,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/siren-head-the...,5
119619,Agent Murphy,2021-01-22,0,1,1,0,[],0.0,"[Adventure, Indie, Simulator]",[Windows PC],...,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/agent-murphy/,5
119625,Escape From Lavender Island,2023-09-01,1,0,4,1,"[Chris Parrello, Jeremy Couillard]",0.0,"[Adventure, Indie, Simulator]","[Windows PC, Linux]",...,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/escape-from-la...,5


In [8]:
def drop_unnecesary_coulumns(df):
    cols_to_drop = ['developers', 'platforms',
                    'ratings_one_zero', 'ratings_one_five', 
                    'ratings_two_zero', 'ratings_two_five', 
                    'ratings_three_zero', 'ratings_three_five', 
                    'ratings_four_zero', 'ratings_four_five',
                    'ratings_five_zero', 'ratings_zero_five',
                    'image', 'url', 'main', 'title', 'release_date']

    return df.drop(cols_to_drop, axis=1)

In [9]:
cleaning = FunctionTransformer(cleaning_in_notebook)
remove_0_reviews = FunctionTransformer(remove_no_reviews)
only_mains = FunctionTransformer(only_main_games)
drop_some_cols = FunctionTransformer(drop_unnecesary_coulumns)



pipeline = Pipeline([('cleaning', cleaning),
                     ('remove_no_reviews', remove_0_reviews), 
                     ('keeps_only_main_games', only_mains), 
                     ('drops_useless_columns', drop_some_cols)])


In [10]:
pipeline.fit_transform(data)

/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column]=pd.to_datetime(df[column])
/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_2_features.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupe['gen'] = None
/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

,plays,playing,backlogs,wishlist,avg_review,genres,description,total_reviews,total_lists,gen
69,7,0,2,2,2.2,[Visual Novel],We all have multiple sides. Pick your poison! ...,1,4,6
166,80,1,82,297,3.3,"[Brawler, Indie, Shooter]",Anger Foot is a lightning-fast hard-bass blast...,8,104,6
377,9,0,5,1,2.4,"[Adventure, RPG, Simulator]",Life is Feudal: MMO is an open-world survival ...,0,3,6
599,10,3,9,41,4.5,"[Adventure, Fighting, Platform]",Play the history. Go behind the scenes of Jord...,5,12,5
634,16,0,26,155,2.1,"[Adventure, Indie, Puzzle]","Become Mia, a paranormal detective searching f...",12,27,6
...,...,...,...,...,...,...,...,...,...,...
118902,4,0,1,4,3.8,[Shooter],"In the year 2991 AD, the Galactic Federal Gove...",0,2,2
119027,6,0,0,1,2.7,"[Adventure, Indie]","Level Up! is a Game created by Titch007uk, and...",1,2,4
119068,3,0,0,0,2.2,[],Sea Battle is a board game everybody loves sin...,0,0,5
119219,18,0,0,0,2.0,"[Indie, Simulator, Strategy]","Control minds, insert subliminal messages into...",2,7,5


In [ ]:
cat_transformer = MultiLabelBinarizer()

In [20]:
ohe, mlb = categorical_encoder(testing_df['genres'])

In [22]:
keep_x_OHE_columns(ohe)

,Adventure,Indie,RPG,Simulator,Strategy,Puzzle,Shooter,Arcade,Platform,
0,0,1,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
108076,0,0,0,0,1,0,0,0,0,0
108077,0,1,0,0,0,0,0,0,0,0
108078,1,1,0,1,0,0,0,0,0,0
108079,1,1,0,1,0,0,0,0,0,0
